In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import dates as mdate
import json
import re
import pymysql
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
#sudo pip3 install ipywidgets ipydatetime
#jupyter nbextension enable --py widgetsnbextension
import ipywidgets as widgets
import datetime
from IPython.display import display, clear_output, Markdown
pd.options.display.float_format = "{:.0f}".format
plt.rcParams['figure.figsize'] = [15,7]
plt.rcParams['timezone'] = 'America/Sao_Paulo'

with open('../../server/app/config-remote.json') as json_data_file:
    data = json.load(json_data_file)
    url = data['SQLALCHEMY_DATABASE_URI']
    mux_port_threshold = data['MUX_PORT_THRESHOLD']
user, password, host, database = re.match('mysql://(.*?):(.*?)@(.*?)/(.*)', url).groups()
conn = pymysql.connect(host=host,port=3306,user=user,passwd=password,db=database)
MOISTURE_NODE_DATA={}

In [ ]:
def get_node_list():
    query_nodes="""
        SELECT  NODE_ID, 
                LAST_UPDATE 
        FROM OASIS_HEARTBEAT
        ORDER BY LAST_UPDATE desc
        """
    nodes = pd.read_sql_query(query_nodes, conn)
    nodes['LAST_UPDATE'] = (pd.to_datetime(nodes['LAST_UPDATE'],unit='s')
                            .dt.tz_localize('utc')
                            .dt.tz_convert('America/Sao_Paulo'))
    return nodes

def get_node_moisture_data(node):
    query="""
        SELECT  TIMESTAMP,
                DATA->'$.DATA.NODE.FREE_HEAP' AS FREE_HEAP,
                DATA->'$.DATA.CAPACITIVEMOISTURE.MUX0' AS MUX0,
                DATA->'$.DATA.CAPACITIVEMOISTURE.MUX1' AS MUX1,
                DATA->'$.DATA.CAPACITIVEMOISTURE.MUX2' AS MUX2,
                DATA->'$.DATA.CAPACITIVEMOISTURE.MUX3' AS MUX3,
                DATA->'$.DATA.CAPACITIVEMOISTURE.MUX4' AS MUX4,
                DATA->'$.DATA.CAPACITIVEMOISTURE.MUX5' AS MUX5,
                DATA->'$.DATA.CAPACITIVEMOISTURE.MUX6' AS MUX6,
                DATA->'$.DATA.CAPACITIVEMOISTURE.MUX7' AS MUX7
        FROM OASIS_DATA
        WHERE NODE_ID='{}'
        ORDER BY TIMESTAMP asc
        """.format(node)
    df = pd.read_sql_query(query, conn).astype(int)
    df.set_index('TIMESTAMP', inplace=True)
    if len(df) == 0:
        print ("get_node_moisture_data: no moisture data found!!!")
        return {}
    else:
        print("Node:", node,
                "Data interval:", int((df.index.max() - df.index.min())/len(df)),"secs",
             ", Duration:", int((df.index.max() - df.index.min())/3600),"hrs",
             ", Memory usage:",int(df.memory_usage(deep=True).sum()/1024),"KB")
    return df
def get_node_temperature_data(node):
    query="""
        SELECT  TIMESTAMP,
                DATA->'$.DATA.DHT.TEMPERATURE' AS TEMPERATURE,
                DATA->'$.DATA.DHT.HUMIDITY' AS HUMIDITY
        FROM OASIS_DATA
        WHERE NODE_ID='oasis-397c0e'
        ORDER BY TIMESTAMP asc
        """
    dft = pd.read_sql_query(query, conn).astype(int)
    return dft

"""

"""
def plot_node_moisture():
    outt = widgets.Output()
    nodes = get_node_list()
    dropdown_node = widgets.Dropdown(options=nodes['NODE_ID'], description='Node:')
    btn_search = widgets.Button(description='Retrieve data',button_style='warning')
    def btn_search_clicked(b):
        global MOISTURE_NODE_DATA
        with outt:
            clear_output()
            MOISTURE_NODE_DATA = get_node_moisture_data(dropdown_node.value)
            if len(MOISTURE_NODE_DATA) != 0:
                rolling.disabled = False
                timerange.disabled = False
                btn_filter_range.disabled = False
                btn_today.disabled = False
                btn_yesterday.disabled = False
                btn_latest_48_hrs.disabled = False
                btn_zones.disabled = False
                for checkbox in mux_checkbox:
                    checkbox.disabled = False

                timerange.min = 0
                timerange.max = 1
                timerange.max = MOISTURE_NODE_DATA.index.max()
                timerange.min = MOISTURE_NODE_DATA.index.min()
                timerange.value = [MOISTURE_NODE_DATA.index.min(), MOISTURE_NODE_DATA.index.max()]
            else:
                rolling.disabled = True
                timerange.disabled = True
                btn_filter_range.disabled = True
                btn_today.disabled = True
                btn_yesterday.disabled = True
                btn_latest_48_hrs.disabled = True
                btn_zones.disabled = True
                for checkbox in mux_checkbox:
                    checkbox.disabled = True

                MOISTURE_NODE_DATA={}
                timerange.min = 0
                timerange.max = 0
                timerange.value = [0, 0]

    btn_search.on_click(btn_search_clicked)
    menu = widgets.HBox([dropdown_node, btn_search])
    
    
    muxes = ['MUX0','MUX1','MUX2','MUX3','MUX4','MUX5','MUX7']
    mux_checkbox = [widgets.Checkbox(description=mux, value=True,indent=False, disabled=True) for mux in muxes]
    mux_checkboxes = widgets.HBox(mux_checkbox)
    rolling = widgets.FloatSlider(
             value=60,
             min=1,
             max=200,
             step=1,
             disabled=True,
             description='Rolling:',
            )
    
    timerange = widgets.IntRangeSlider(
            value=[0,0],
            min=0,
            max=0,
            step=5,
            description='Range:',
            disabled=True,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            layout=dict(width='95%')
            )

    btn_filter_range = widgets.Button(description='Filter Range',button_style='danger', disabled=True)
    btn_today = widgets.Button(description='Today',button_style='info', disabled=True)
    btn_yesterday = widgets.Button(description='Yesterday',button_style='success', disabled=True)
    btn_latest_48_hrs = widgets.Button(description='Latest 48 hrs',button_style='warning', disabled=True)
    btn_zones = widgets.ToggleButton(
                value=True,
                description='Moisture Level',
                disabled=True,
                tooltip='Show wet and dry zones',
            )
    buttons = widgets.HBox((btn_filter_range, btn_today, btn_yesterday, btn_latest_48_hrs, btn_zones))


    def plot_time_range(start, end, hrs_only):
        with outt:
            clear_output()
            print("Start time:", datetime.datetime.fromtimestamp(start).strftime('%Y-%m-%d %H:%M:%S'))
            print("  End time:", datetime.datetime.fromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S'))
            selected_muxes = [checkbox.description for checkbox in mux_checkbox if checkbox.value]

            data = MOISTURE_NODE_DATA[(MOISTURE_NODE_DATA.index > start) & 
                (MOISTURE_NODE_DATA.index < end)][selected_muxes].rolling(int(rolling.value)).mean().dropna()

            plt.rcParams['figure.figsize'] = [15,7]
            plt.rcParams['timezone'] = 'America/Sao_Paulo'

            fig,ax = plt.subplots()
            x = mdate.epoch2num(data.index)
            fmt = mdate.DateFormatter('%H:%M' if hrs_only else '%y-%m-%d %H:%M')

            ax.xaxis.set_major_formatter(fmt)
            plt.xticks( rotation=25 )
            plt.plot_date(x, data, linestyle='solid', marker='None')
            if btn_zones.value:
                plt.fill_between(x, mux_port_threshold['OFFLINE'], mux_port_threshold['WET'],
                                 where=mux_port_threshold['WET'] > mux_port_threshold['OFFLINE'],
                        facecolor='blue', alpha=0.2)
                plt.fill_between(x, mux_port_threshold['WET'], mux_port_threshold['NOSOIL'],
                         where=mux_port_threshold['NOSOIL'] > mux_port_threshold['WET'],
                facecolor='red', alpha=0.2)
            plt.legend(selected_muxes)
            plt.show()
    def on_btn_filter_range_clicked(b):
        plot_time_range(timerange.value[0], timerange.value[1], False)
    def on_btn_today(b):
        now = datetime.datetime.today()
        start_day = datetime.datetime(now.year,now.month,now.day,0,0,0)
        plot_time_range(start_day.timestamp(), now.timestamp(), True)
    def on_btn_yesterday(b):
        yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
        start_day = datetime.datetime(yesterday.year,yesterday.month,yesterday.day,0,0,0)
        end_day = datetime.datetime(yesterday.year,yesterday.month,yesterday.day,23,59,59)
        plot_time_range(start_day.timestamp(), end_day.timestamp(), True)
    def on_btn_latest_48_hours(b):
        now = datetime.datetime.today()
        latest_48_hours = now - datetime.timedelta(hours=48)
        plot_time_range(latest_48_hours.timestamp(), now.timestamp(), False)

    btn_filter_range.on_click(on_btn_filter_range_clicked)
    btn_today.on_click(on_btn_today)
    btn_yesterday.on_click(on_btn_yesterday)
    btn_latest_48_hrs.on_click(on_btn_latest_48_hours)

    return widgets.VBox([menu, mux_checkboxes, rolling, timerange, buttons, outt])

In [ ]:
plot_node_moisture()